In [64]:

import argparse
import os
import sys
import json
import random

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import nltk

import pandas as pd

In [10]:
PAD_TOKEN = '_PAD_'
UNK_TOKEN = '_UNK_'


mydir = os.path.dirname(os.path.abspath('data'))

In [11]:
mydir

'/home/adrien/Documents/CS5304-DSW/assignments/assignment4'

In [61]:
# Models

class BagOfWordsModel(nn.Module):
  def __init__(self, embeddings):
    super(BagOfWordsModel, self).__init__()
    
    print(embeddings)
    
    self.embed = nn.Embedding(embeddings.shape[0], embeddings.shape[1], sparse=True)
    self.embed.weight.data.copy_(torch.from_numpy(embeddings))
    self.classify = nn.Linear(embeddings.shape[1], 5)

  def forward(self, x):
    return self.classify(self.embed(x).sum(1))


class CNNClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, output_size, kernel_dim=100, kernel_sizes=(3, 4, 5), dropout=0.5):
        super(CNNClassifier,self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(1, kernel_dim, (K, embedding_dim)) for K in kernel_sizes])

        # kernal_size = (K,D) 
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(kernel_sizes) * kernel_dim, output_size)
    
    
    def init_weights(self, pretrained_word_vectors, is_static=False):
        self.embedding.weight = nn.Parameter(torch.from_numpy(pretrained_word_vectors).float())
        if is_static:
            self.embedding.weight.requires_grad = False


    def forward(self, inputs, is_training=False):
        inputs = self.embedding(inputs).unsqueeze(1) # (B,1,T,D)
        inputs = [F.relu(conv(inputs)).squeeze(3) for conv in self.convs] #[(N,Co,W), ...]*len(Ks)
        inputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in inputs] #[(N,Co), ...]*len(Ks)

        concated = torch.cat(inputs, 1)

        if is_training:
            concated = self.dropout(concated) # (N,len(Ks)*Co)
        out = self.fc(concated) 
        return F.log_softmax(out,1)
    
    

In [13]:
class TokenConverter(object):
  def __init__(self, vocab):
    self.vocab = vocab
    self.unknown = 0

  def convert(self, token):
    if token in self.vocab:
      id = self.vocab.get(token)
    else:
      id = self.vocab.get(UNK_TOKEN)
      self.unknown += 1
    return id

In [14]:
# Methods for loading SST data

def sentiment2label(x):
  if x >= 0 and x <= 0.2:
    return 0
  elif x > 0.2 and x <= 0.4:
    return 1
  elif x > 0.4 and x <= 0.6:
    return 2
  elif x > 0.6 and x <= 0.8:
    return 3
  elif x > 0.8 and x <= 1:
    return 4
  else:
    raise ValueError('Improper sentiment value {}'.format(x))

In [15]:
def read_dictionary_txt_with_phrase_ids(dictionary_path, phrase_ids_path, labels_path=None):
  print('Reading data dictionary_path={} phrase_ids_path={} labels_path={}'.format(
    dictionary_path, phrase_ids_path, labels_path))

  with open(phrase_ids_path) as f:
    phrase_ids = set(line.strip() for line in f)

  with open(dictionary_path) as f:
    examples_dict = dict()
    for line in f:
      parts = line.strip().split('|')
      phrase = parts[0]
      phrase_id = parts[1]

      if phrase_id not in phrase_ids:
        continue

      example = dict()
      example['phrase'] = phrase.replace('(', '-LRB').replace(')', '-RRB-')
      example['tokens'] = example['phrase'].split(' ')
      example['example_id'] = phrase_id
      example['label'] = None
      examples_dict[example['example_id']] = example

  if labels_path is not None:
    with open(labels_path) as f:
      for i, line in enumerate(f):
        if i == 0:
          continue
        parts = line.strip().split('|')
        phrase_id = parts[0]
        sentiment = float(parts[1])
        label = sentiment2label(sentiment)

        if phrase_id in examples_dict:
          examples_dict[phrase_id]['label'] = label

  examples = [ex for _, ex in examples_dict.items()]

  print('Found {} examples.'.format(len(examples)))

  return examples

In [16]:
def build_vocab(datasets):
  vocab = dict()
  vocab[PAD_TOKEN] = len(vocab)
  vocab[UNK_TOKEN] = len(vocab)
  for data in datasets:
    for example in data:
      for word in example['tokens']:
        if word not in vocab:
          vocab[word] = len(vocab)

  print('Vocab size: {}'.format(len(vocab)))

  return vocab

In [17]:
def convert2ids(data, vocab):
  converter = TokenConverter(vocab)
  for example in data:
    example['tokens'] = list(map(converter.convert, example['tokens']))
  print('Found {} unknown tokens.'.format(converter.unknown))
  return data

In [18]:
def load_data_and_embeddings(data_path, phrase_ids_path, embeddings_path):
  labels_path = os.path.join(data_path, 'sentiment_labels.txt')
  dictionary_path = os.path.join(data_path, 'dictionary.txt')
  train_data = read_dictionary_txt_with_phrase_ids(dictionary_path, os.path.join(phrase_ids_path, 'phrase_ids.train.txt'), labels_path)
  validation_data = read_dictionary_txt_with_phrase_ids(dictionary_path, os.path.join(phrase_ids_path, 'phrase_ids.dev.txt'), labels_path)
  test_data = read_dictionary_txt_with_phrase_ids(dictionary_path, os.path.join(phrase_ids_path, 'phrase_ids.test.txt'))
  vocab = build_vocab([train_data, validation_data, test_data])
  vocab, embeddings = load_embeddings(options.embeddings, vocab, cache=True)
  train_data = convert2ids(train_data, vocab)
  validation_data = convert2ids(validation_data, vocab)
  test_data = convert2ids(test_data, vocab)
  return train_data, validation_data, test_data, vocab, embeddings

In [19]:
def load_embeddings(path, vocab, cache=False, cache_path=None):
  rows = []
  new_vocab = [UNK_TOKEN]

  if cache_path is None:
    cache_path = path + '.cache'

  # Use cache file if it exists.
  if os.path.exists(cache_path):
    path = cache_path

  print("Reading embeddings from {}".format(path))

  # first pass over the embeddings to vocab and relevant rows
  with open(path) as f:
    for line in f:
      word, row = line.split(' ', 1)
      if word == UNK_TOKEN:
        raise ValueError('The unk token should not exist w.in embeddings.')
      if word in vocab:
        rows.append(line)
        new_vocab.append(word)

  # optionally save relevant rows to cache file.
  if cache and not os.path.exists(cache_path):
    with open(cache_path, 'w') as f:
      for line in rows:
        f.write(line)
      print("Cached embeddings to {}".format(cache_path))

  # turn vocab list into a dictionary
  new_vocab = {w: i for i, w in enumerate(new_vocab)}

  print('New vocab size: {}'.format(len(new_vocab)))

  assert len(rows) == len(new_vocab) - 1

  # create embeddings matrix
  embeddings = np.zeros((len(new_vocab), 300), dtype=np.float32)
  for i, line in enumerate(rows):
    embeddings[i+1] = list(map(float, line.strip().split(' ')[1:]))

  return new_vocab, embeddings

In [20]:
# Batch Iterator

def prepare_data(data):
  # pad data
  maxlen = max(map(len, data))
  data = [ex + [0] * (maxlen-len(ex)) for ex in data]

  # wrap in tensor
  return torch.LongTensor(data)

In [21]:
def prepare_labels(labels):
  try:
    return torch.LongTensor(labels)
  except:
    return labels


In [22]:
def batch_iterator(dataset, batch_size, forever=False):
  dataset_size = len(dataset)
  order = None
  nbatches = dataset_size // batch_size

  def init_order():
    return random.sample(range(dataset_size), dataset_size)

  def get_batch(start, end):
    batch = [dataset[ii] for ii in order[start:end]]
    data = prepare_data([ex['tokens'] for ex in batch])
    labels = prepare_labels([ex['label'] for ex in batch])
    example_ids = [ex['example_id'] for ex in batch]
    return data, labels, example_ids

  order = init_order()

  while True:
    for i in range(nbatches):
      start = i*batch_size
      end = (i+1)*batch_size
      yield get_batch(start, end)

    if nbatches*batch_size < dataset_size:
      yield get_batch(nbatches*batch_size, dataset_size)

    if not forever:
      break
    
    order = init_order()

In [23]:
# Utility Methods

def checkpoint_model(step, val_err, model, opt, save_path):
  save_dict = dict(
    step=step,
    val_err=val_err,
    model_state_dict=model.state_dict(),
    opt_state_dict=opt.state_dict())
  torch.save(save_dict, save_path)


def load_model(model, opt, load_path):
  load_dict = torch.load(load_path)
  step = load_dict['step']
  val_err = load_dict['val_err']
  model.load_state_dict(load_dict['model_state_dict'])
  opt.load_state_dict(load_dict['opt_state_dict'])
  return step, val_err

In [63]:
# Main

def run_validation(model, dataset, options):
  err = 0
  count = 0
  for data, labels, _ in batch_iterator(dataset, options.batch_size, forever=False):
    outp = model(Variable(data))
    loss = nn.NLLLoss()(F.log_softmax(outp), Variable(labels))
    acc = (outp.data.max(1)[1] == labels).sum() / data.shape[0]
    err += (1-acc) * data.shape[0]
    count += data.shape[0]
  err = err / count
  print('Ev-Err={}'.format(err))
  return err


def run_test(model, dataset, options):
  print('Writing predictions to {}'.format(os.path.abspath(options.predictions)))

  preds_dict = dict()

  for data, _, example_ids in batch_iterator(dataset, options.batch_size, forever=False):
    outp = model(Variable(data))
    preds = outp.data.max(1)[1]

    for id, pred in zip(example_ids, preds):
      preds_dict[id] = pred

  with open(options.predictions, 'w') as f:
    for id, pred in preds_dict.items():
      f.write('{}|{}\n'.format(id, pred))


def run(options):
  train_data, validation_data, test_data, vocab, embeddings = \
    load_data_and_embeddings(options.data, options.ids, options.embeddings)
  model = BagOfWordsModel(embeddings) # change this if we create a different class names than bag of words
  opt = optim.SGD(model.parameters(), lr=3e-4)
  
  step = 0
  best_val_err = 1

  if options.eval_only_mode:
    step, best_val_err = load_model(model, opt, options.model)
    print('Model loaded from {}\nstep={} best_val_err={}'.format(options.model, step, best_val_err))
    run_test(model, test_data, options)
    sys.exit()
  
  for data, labels, _ in batch_iterator(train_data, options.batch_size, forever=True):
    outp = model(Variable(data))
    loss = nn.NLLLoss()(F.log_softmax(outp), Variable(labels))
    acc = (outp.data.max(1)[1] == labels).sum() / data.shape[0]

    opt.zero_grad()
    loss.backward()
    opt.step()
    
    if step % options.log_every == 0:
      print('Step={} Tr-Loss={} Tr-Acc={}'.format(step, loss.data[0], acc))
      
    if step % options.eval_every == 0:
      val_err = run_validation(model, validation_data, options)
      
      # early stopping
      if val_err < best_val_err:
        best_val_err = val_err
        print('Checkpointing model step={} best_val_err={}.'.format(step, best_val_err))
        checkpoint_model(step, val_err, model, opt, options.model)
    
    step += 1


if __name__ == '__main__':
#   parser = argparse.ArgumentParser()
#   parser.add_argument('--ids', default=mydir, type=str)
#   parser.add_argument('--data', default=os.path.expanduser('~/data/stanfordSentimentTreebank'), type=str)
#   parser.add_argument('--embeddings', default=, type=str)
#   parser.add_argument('--model', default=os.path.join(mydir, 'model.ckpt'), type=str)
#   parser.add_argument('--predictions', default=os.path.join(mydir, 'predictions.txt'), type=str)
#   parser.add_argument('--log_every', default=100, type=int)
#   parser.add_argument('--eval_every', default=1000, type=int)
#   parser.add_argument('--batch_size', default=32, type=int)
#   parser.add_argument('--eval_only_mode', action='store_true')
#   options = parser.parse_args()

#   print(json.dumps(options.__dict__, sort_keys=True, indent=4))
    
    options = Namespace(
        ids=mydir,
        data=os.path.expanduser(mydir + '/data/stanfordSentimentTreebank'),
        embeddings = os.path.expanduser(mydir +'/data/glove/glove.840B.300d.txt'),
        model = os.path.join(mydir, 'model.ckpt'),
        predictions = os.path.join(mydir, 'predictions.txt'),
        log_every = 100,
        batch_size = 32,
        eval_only_mode = False,
        eval_every=1000
    )
    
    
#     {
#         ids:mydir,
#         data: os.path.expanduser(mydir + '/data/stanfordSentimentTreebank'),
#         'embeddings':0
#     }
    print(options)
    
    run(options)


Reading data dictionary_path=/home/adrien/Documents/CS5304-DSW/assignments/assignment4/data/stanfordSentimentTreebank/dictionary.txt phrase_ids_path=/home/adrien/Documents/CS5304-DSW/assignments/assignment4/phrase_ids.train.txt labels_path=/home/adrien/Documents/CS5304-DSW/assignments/assignment4/data/stanfordSentimentTreebank/sentiment_labels.txt
Found 159274 examples.
Reading data dictionary_path=/home/adrien/Documents/CS5304-DSW/assignments/assignment4/data/stanfordSentimentTreebank/dictionary.txt phrase_ids_path=/home/adrien/Documents/CS5304-DSW/assignments/assignment4/phrase_ids.dev.txt labels_path=/home/adrien/Documents/CS5304-DSW/assignments/assignment4/data/stanfordSentimentTreebank/sentiment_labels.txt
Found 24772 examples.
Reading data dictionary_path=/home/adrien/Documents/CS5304-DSW/assignments/assignment4/data/stanfordSentimentTreebank/dictionary.txt phrase_ids_path=/home/adrien/Documents/CS5304-DSW/assignments/assignment4/phrase_ids.test.txt labels_path=None
Found 46663 e

/home/adrien/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/home/adrien/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


Step=0 Tr-Loss=2.099637746810913 Tr-Acc=0.0625
Ev-Err=0.8734861940901018
Checkpointing model step=0 best_val_err=0.8734861940901018.
Step=100 Tr-Loss=1.53522527217865 Tr-Acc=0.4375
Step=200 Tr-Loss=1.4813640117645264 Tr-Acc=0.4375
Step=300 Tr-Loss=1.27640700340271 Tr-Acc=0.625
Step=400 Tr-Loss=1.2276989221572876 Tr-Acc=0.625
Step=500 Tr-Loss=1.4000253677368164 Tr-Acc=0.40625
Step=600 Tr-Loss=1.2505191564559937 Tr-Acc=0.59375
Step=700 Tr-Loss=1.1712247133255005 Tr-Acc=0.5
Step=800 Tr-Loss=1.2568507194519043 Tr-Acc=0.46875
Step=900 Tr-Loss=1.143091082572937 Tr-Acc=0.53125
Step=1000 Tr-Loss=1.290806531906128 Tr-Acc=0.5
Ev-Err=0.44618924592281606
Checkpointing model step=1000 best_val_err=0.44618924592281606.
Step=1100 Tr-Loss=1.1379947662353516 Tr-Acc=0.5625
Step=1200 Tr-Loss=1.6393976211547852 Tr-Acc=0.375
Step=1300 Tr-Loss=1.2306565046310425 Tr-Acc=0.5
Step=1400 Tr-Loss=1.311489224433899 Tr-Acc=0.46875
Step=1500 Tr-Loss=1.1129697561264038 Tr-Acc=0.53125
Step=1600 Tr-Loss=1.1075829267501

Step=14200 Tr-Loss=0.8410071730613708 Tr-Acc=0.6875
Step=14300 Tr-Loss=0.8845726251602173 Tr-Acc=0.65625
Step=14400 Tr-Loss=1.147903323173523 Tr-Acc=0.5625
Step=14500 Tr-Loss=1.1379666328430176 Tr-Acc=0.53125
Step=14600 Tr-Loss=1.063138723373413 Tr-Acc=0.6875
Step=14700 Tr-Loss=0.9914287328720093 Tr-Acc=0.65625
Step=14800 Tr-Loss=0.950739860534668 Tr-Acc=0.625
Step=14900 Tr-Loss=1.0611082315444946 Tr-Acc=0.5625
Step=15000 Tr-Loss=0.9058551788330078 Tr-Acc=0.65625
Ev-Err=0.3885031487162926
Checkpointing model step=15000 best_val_err=0.3885031487162926.
Step=15100 Tr-Loss=0.7597969770431519 Tr-Acc=0.65625
Step=15200 Tr-Loss=1.1763336658477783 Tr-Acc=0.46875
Step=15300 Tr-Loss=1.0992035865783691 Tr-Acc=0.53125
Step=15400 Tr-Loss=1.049932837486267 Tr-Acc=0.46875
Step=15500 Tr-Loss=0.8321266174316406 Tr-Acc=0.625
Step=15600 Tr-Loss=1.0403918027877808 Tr-Acc=0.59375
Step=15700 Tr-Loss=1.1172641515731812 Tr-Acc=0.65625
Step=15800 Tr-Loss=1.1794202327728271 Tr-Acc=0.5625
Step=15900 Tr-Loss=1.0

Step=28800 Tr-Loss=1.0007222890853882 Tr-Acc=0.75
Step=28900 Tr-Loss=1.1199853420257568 Tr-Acc=0.59375
Step=29000 Tr-Loss=0.9954856038093567 Tr-Acc=0.625
Ev-Err=0.37994509930566767
Checkpointing model step=29000 best_val_err=0.37994509930566767.
Step=29100 Tr-Loss=0.9962900280952454 Tr-Acc=0.6875
Step=29200 Tr-Loss=1.0607411861419678 Tr-Acc=0.53125
Step=29300 Tr-Loss=1.2018908262252808 Tr-Acc=0.40625
Step=29400 Tr-Loss=1.0463414192199707 Tr-Acc=0.5625
Step=29500 Tr-Loss=1.035245418548584 Tr-Acc=0.53125
Step=29600 Tr-Loss=1.0425055027008057 Tr-Acc=0.625
Step=29700 Tr-Loss=1.023579716682434 Tr-Acc=0.65625
Step=29800 Tr-Loss=0.8906805515289307 Tr-Acc=0.625
Step=29900 Tr-Loss=0.9311282634735107 Tr-Acc=0.59375
Step=30000 Tr-Loss=0.9922327399253845 Tr-Acc=0.5625
Ev-Err=0.38115614403358633
Step=30100 Tr-Loss=1.469195008277893 Tr-Acc=0.53125
Step=30200 Tr-Loss=1.4914281368255615 Tr-Acc=0.375
Step=30300 Tr-Loss=0.7869265079498291 Tr-Acc=0.78125
Step=30400 Tr-Loss=0.9295481443405151 Tr-Acc=0.687

...

Step=1398900 Tr-Loss=0.8449333906173706 Tr-Acc=0.625
Step=1399000 Tr-Loss=0.8107852935791016 Tr-Acc=0.6875
Ev-Err=0.3611335378653318
Step=1399100 Tr-Loss=1.1041748523712158 Tr-Acc=0.5
Step=1399200 Tr-Loss=0.8741056323051453 Tr-Acc=0.75
Step=1399300 Tr-Loss=1.0714645385742188 Tr-Acc=0.53125
Step=1399400 Tr-Loss=1.070660948753357 Tr-Acc=0.625
Step=1399500 Tr-Loss=1.0806686878204346 Tr-Acc=0.625
Step=1399600 Tr-Loss=0.6411904692649841 Tr-Acc=0.78125
Step=1399700 Tr-Loss=1.0173248052597046 Tr-Acc=0.625
Step=1399800 Tr-Loss=0.7462269067764282 Tr-Acc=0.78125
Step=1399900 Tr-Loss=0.8831422328948975 Tr-Acc=0.65625
Step=1400000 Tr-Loss=0.7574469447135925 Tr-Acc=0.71875
Ev-Err=0.362142741805264
Step=1400100 Tr-Loss=0.8233099579811096 Tr-Acc=0.65625
Step=1400200 Tr-Loss=0.8211250901222229 Tr-Acc=0.65625
Step=1400300 Tr-Loss=0.9307336807250977 Tr-Acc=0.59375
Step=1400400 Tr-Loss=1.2275906801223755 Tr-Acc=0.625
Step=1400500 Tr-Loss=1.0285478830337524 Tr-Acc=0.5
Step=1400600 Tr-Loss=0.99836868047714

Step=1413400 Tr-Loss=0.984900712966919 Tr-Acc=0.625
Step=1413500 Tr-Loss=1.0967410802841187 Tr-Acc=0.625
Step=1413600 Tr-Loss=0.8482981324195862 Tr-Acc=0.625
Step=1413700 Tr-Loss=1.026993989944458 Tr-Acc=0.59375
Step=1413800 Tr-Loss=0.9238336682319641 Tr-Acc=0.5625
Step=1413900 Tr-Loss=0.8054981827735901 Tr-Acc=0.75
Step=1414000 Tr-Loss=0.7685307264328003 Tr-Acc=0.65625
Ev-Err=0.36040691102858063
Step=1414100 Tr-Loss=0.8318665027618408 Tr-Acc=0.65625
Step=1414200 Tr-Loss=1.282960057258606 Tr-Acc=0.46875
Step=1414300 Tr-Loss=0.7819551825523376 Tr-Acc=0.6875
Step=1414400 Tr-Loss=0.9671393036842346 Tr-Acc=0.5625
Step=1414500 Tr-Loss=0.8327969312667847 Tr-Acc=0.625
Step=1414600 Tr-Loss=0.754744827747345 Tr-Acc=0.65625
Step=1414700 Tr-Loss=0.9723515510559082 Tr-Acc=0.65625
Step=1414800 Tr-Loss=0.780712902545929 Tr-Acc=0.65625
Step=1414900 Tr-Loss=0.9631987810134888 Tr-Acc=0.59375
Step=1415000 Tr-Loss=0.826149046421051 Tr-Acc=0.59375
Ev-Err=0.36190053285968027
Step=1415100 Tr-Loss=0.64591705

Step=1428000 Tr-Loss=1.4003238677978516 Tr-Acc=0.53125
Ev-Err=0.36484740836428226
Step=1428100 Tr-Loss=0.9414730668067932 Tr-Acc=0.625
Step=1428200 Tr-Loss=1.0674504041671753 Tr-Acc=0.59375
Step=1428300 Tr-Loss=0.8383100628852844 Tr-Acc=0.65625
Step=1428400 Tr-Loss=1.1712532043457031 Tr-Acc=0.5625
Step=1428500 Tr-Loss=0.9928439259529114 Tr-Acc=0.59375
Step=1428600 Tr-Loss=0.962716817855835 Tr-Acc=0.6875
Step=1428700 Tr-Loss=0.9389837384223938 Tr-Acc=0.5625
Step=1428800 Tr-Loss=0.837056577205658 Tr-Acc=0.5625
Step=1428900 Tr-Loss=0.7793413996696472 Tr-Acc=0.75
Step=1429000 Tr-Loss=0.769756019115448 Tr-Acc=0.625
Ev-Err=0.3661391894073954
Step=1429100 Tr-Loss=0.989619791507721 Tr-Acc=0.59375
Step=1429200 Tr-Loss=1.0571465492248535 Tr-Acc=0.53125
Step=1429300 Tr-Loss=0.8429928421974182 Tr-Acc=0.65625
Step=1429400 Tr-Loss=1.0185859203338623 Tr-Acc=0.59375
Step=1429500 Tr-Loss=0.9411507248878479 Tr-Acc=0.6875
Step=1429600 Tr-Loss=0.8552691340446472 Tr-Acc=0.625
Step=1429700 Tr-Loss=0.8313006

Step=1442400 Tr-Loss=1.2851756811141968 Tr-Acc=0.5
Step=1442500 Tr-Loss=0.8543478846549988 Tr-Acc=0.71875
Step=1442600 Tr-Loss=0.9077231287956238 Tr-Acc=0.59375
Step=1442700 Tr-Loss=0.8070639371871948 Tr-Acc=0.5625
Step=1442800 Tr-Loss=0.7886319756507874 Tr-Acc=0.6875
Step=1442900 Tr-Loss=0.9405122399330139 Tr-Acc=0.59375
Step=1443000 Tr-Loss=1.14374577999115 Tr-Acc=0.65625
Ev-Err=0.3606491199741644
Step=1443100 Tr-Loss=1.2515549659729004 Tr-Acc=0.5
Step=1443200 Tr-Loss=1.1412453651428223 Tr-Acc=0.53125
Step=1443300 Tr-Loss=0.9636183977127075 Tr-Acc=0.625
Step=1443400 Tr-Loss=1.2821621894836426 Tr-Acc=0.5625
Step=1443500 Tr-Loss=0.8916954398155212 Tr-Acc=0.71875
Step=1443600 Tr-Loss=0.9216358065605164 Tr-Acc=0.53125
Step=1443700 Tr-Loss=0.8731346130371094 Tr-Acc=0.625
Step=1443800 Tr-Loss=0.818215012550354 Tr-Acc=0.6875
Step=1443900 Tr-Loss=0.7575731873512268 Tr-Acc=0.6875
Step=1444000 Tr-Loss=0.645361602306366 Tr-Acc=0.8125
Ev-Err=0.3641207815275311
Step=1444100 Tr-Loss=1.156799077987

KeyboardInterrupt: 

In [48]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [ ]:
%tb